In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.special import expit
from scipy.optimize import fmin, minimize
import glob as glob
import matplotlib
matplotlib.style.use('ggplot')

# .py
%load_ext autoreload
%autoreload 2
import weather_alphas, weather_costs, model, model_01, validate, prediction

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


running model
getting cost from nweek ahead
getting bounded alphas ...
done getting alphas ...
running model


In [2]:
def split_data(data):
    train_mask = (data.index < "2011")
    train = data[train_mask]
    test_mask = (data.index > "2011")
    test = data[test_mask]
    return train, test

In [3]:
all_weather = pd.read_csv("weather01-16.csv")

In [4]:
cm_weather = all_weather[all_weather.stn_name == "Chiang Mai"]
cm_weather.index = pd.DatetimeIndex(cm_weather.date)
mask = (cm_weather.index > "2003") & (cm_weather.index < "2016")
cm_weather = cm_weather[mask]

provinces_df = pd.read_csv("all-dengues.csv")
provinces_df.index = pd.DatetimeIndex(provinces_df.date)
provinces_df = provinces_df.drop(['date','date.1'],axis=1)
cm_dengues = provinces_df[provinces_df['จังหวัด'] == 'ChiangMai'].resample('W').size()
cm_dengues_df = pd.DataFrame(cm_dengues,columns=['cases'])

In [5]:
cm_avg_weather = cm_weather[['avgrh','dday','meantemp']].resample('W').mean()
cm_avg_weather['rain'] = cm_weather[['rain']].resample('W').sum() # cumulative rainfall
cm_dengues_weather = pd.concat([cm_avg_weather,cm_dengues_df[:-52]],axis=1)

cm_dengues_weather_split = split_data(cm_dengues_weather)
cm_dengues_train, cm_dengues_test = cm_dengues_weather_split[0], cm_dengues_weather_split[1]

In [6]:
def make_line(start_week,real,predictions,title,x_axis,y_axis,
              real_legend='Real',predict_legend='Prediction'):
    plt.figure(figsize=(12,6))
    plt.plot(range(len(real)),real,'-r',label=real_legend)
    plt.plot(range(start_week,len(predictions)+start_week),predictions,'-k',label=predict_legend,alpha=0.4)
    plt.axvline(start_week)
    plt.text((start_week*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)
    plt.legend()
    plt.title(title)
    
def plot_all_predictions(LAG, real, province, nweeks_to_predict, all_predictions):
    
    plt.figure(figsize=(13,6))
    plt.plot(range(len(real)),real,label='real')
    graph_title = "%s's Dengue Prediction with Mean Temperature"%province
    plt.title(graph_title)
    for i in range(len(all_predictions)):
        cur_week_guess = nweeks_to_predict[i]
        start_predict_week = LAG+(cur_week_guess-1)
        predicted_cases = all_predictions[i]
        x_weeks = range(start_predict_week,len(predicted_cases)+start_predict_week)
        line_label = "%d week guess"%cur_week_guess
        plt.plot(x_weeks,predicted_cases,label=line_label,alpha=0.8)
    
    seperate_train_test(cm_dengues_df.cases,cm_dengues_train)
    plt.axvline(LAG)
    plt.text((LAG*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel("# week starting 2003")
    plt.ylabel("Dengue cases")
    plt.legend()
    plt.savefig("16-lags-16-weeks-9-temp-wks-chiangmai-00.png")
    
def seperate_train_test(all_data,train_data):
    plt.axvspan(0, len(train_data), color='blue', alpha=0.1)
    plt.text((len(train_data)/2)-25,700,'Train Period',size=16)
    plt.axvline(len(train_data)+1,linewidth=0.5, color='green')
    plt.text(len(train_data)+20,700,'Test Period',size=16)
    plt.axvspan(len(train_data)+1, len(all_data), color='green', alpha=0.1)

In [47]:
import multiprocessing as mp
from multiprocessing import Process, Queue

# Define an output queue
output = Queue()

# def validate

# def compute_ret(temp):
#     for i in range(int(1e5)): pass
#     return temp/2.


ws_csv = np.array(pd.read_csv("best-for-19-lags-ws-3rd-iter.csv").T)[0]
    
def validation(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, train, real, output):
                                    # get_alphas takes 4 args, if theres no input csv
    alphas = weather_alphas.get_alphas(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, train, ws_csv)
                                    # get_predictions takes 6 args
    predictions_to_validate = prediction.get_predictions(LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, real, alphas.x, "CM")
                                    # get_validations takes 3 argsg
    validated_result = validate.get_validations(LAG, predictions_to_validate, real)
    output.put((TEMPERATURE_WEEKS,validated_result,alphas.x)) # add in ws as well alphas.x


In [48]:
%%time
# Setup a list of processes that we want to run
train = cm_dengues_train
real = cm_dengues_weather
processes = [Process(target=validation, args=(19, weeks, 10, train, real, output)) for weeks in range(6,15)]
print ws_csv

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

[  6.33935672e-01   1.67666287e-08   2.31639456e-08   3.50209373e-08
   0.00000000e+00   4.31120912e-03   1.34822610e-01   5.01089508e-04
   5.38066447e-05   1.03042846e-05   7.44309154e-07   1.09314887e-01
   1.38345504e-06   1.11136595e-05   1.54229262e-06   1.94576579e-07
   9.57191545e-08   6.70262672e-08   2.86149822e-02   5.53266735e-01
   1.23621866e+00   9.96473843e+00   2.57910558e-01   3.72814121e-01
   6.90528824e-02  -7.37570214e-02   9.27721001e-04  -6.53165618e-02
   1.08420525e-01   1.45659515e-03  -8.82658404e-02  -3.66435643e-01
  -2.32713124e-03  -1.06584138e-02  -4.24314530e-03   3.08463440e-03
   8.57890639e-03   1.10691040e-02   2.86555863e-03   4.52966854e-02
  -7.53315559e-02  -9.34975403e-02   7.47375805e-02]
40
bnds 40
ws_csv
prev_ws 40
41
42
bnds 41
bnds 42
ws_csv
ws_csv
prev_ws 42
43
prev_ws 41
bnds 43
ws_csv
prev_ws 43
44
bnds 44
ws_csv
prev_ws 44
45
bnds 45
ws_csv
prev_ws 45
46
47
bnds 46
ws_csv
prev_ws 46
48
bnds 48
ws_csv
prev_ws 48
bnds 47
ws_csv
prev_ws

In [57]:
for i in results:
    print len(i[2]) - 20 - 2 - 11 - 1, i[1]
    

13 [162.33996465922505, 277.78484938195356, 540.86711284380294, 1015.9127654623079, 1545.9587818194202]
11 [159.78987951936716, 271.32711040540067, 525.37503724451301, 987.73444024001299, 1535.828820118503]
8 [162.92185456363094, 281.04066781033919, 543.20284646929395, 1000.7868582314842, 1502.409551103457]
7 [207.19334380097658, 376.93441387239932, 713.58745612674079, 1215.4521194714687, 1510.9232939370929]
14 [157.42938544748338, 263.30886540716614, 502.14464603954417, 930.29241863948948, 1490.2338830962601]
12 [163.83093023182298, 282.60623044747013, 551.94273740936819, 1032.6172009628172, 1532.8524337873098]
10 [155.8040202620416, 261.57921647238328, 484.98085705557293, 839.3306770679186, 1351.0650382726158]
9 [200.0629174449127, 352.23836020593444, 669.33723826612652, 1185.2730333989746, 1514.7236564288223]
6 [222.56843121385583, 404.78821271921151, 758.00787029178957, 1251.1288641805734, 1498.5969693846041]


In [49]:
def get_best(validations):
    idx = None
    best_so_far = validations[0][1][-1]
    print best_so_far
    for i in range(len(validations)):
        cur_validation = validations[i][1]
#         print cur_validation
        if cur_validation[0] <= best_so_far:
            best_so_far = cur_validation[0]
            idx = i
    return validations[idx]
        
    
best_param = get_best(results)

1545.95878182


In [50]:
best_param

(19,
 [155.8040202620416,
  261.57921647238328,
  484.98085705557293,
  839.3306770679186,
  1351.0650382726158],
 array([  6.92649393e-01,   2.64375655e-08,   5.57107205e-03,
          5.85047894e-03,   1.16081014e-08,   1.40462654e-09,
          6.56175947e-02,   7.92904587e-03,   0.00000000e+00,
          3.62144859e-10,   3.47953688e-08,   0.00000000e+00,
          5.38318548e-02,   6.31941352e-02,   5.09969092e-02,
          8.85541221e-04,   4.43060414e-04,   1.99637046e-06,
          1.87447580e-07,   4.85597202e-01,   1.62345994e+00,
          7.76065455e+00,   1.45670780e-01,   1.66752571e-01,
         -4.98325611e-02,  -1.22210805e-01,  -1.24426200e-01,
         -1.45793597e-01,  -9.71322795e-02,  -4.53504205e-02,
          4.54247461e-02,   5.24437100e-02,   2.22832017e-01,
         -6.84509777e-01,   1.36653969e-02,   1.38792365e-02,
          1.37320347e-02,   1.11718701e-02,   1.72623793e-03,
         -1.63124482e-02,   1.12192347e-02,   9.28513477e-03,
         -7.724474

In [56]:
pd.DataFrame(best_param[2]).to_csv("best-with-19-lags-11-temps-ws-1st-iter.csv",index=False)
# np.array(pd.read_csv("best-for-19-lags-ws-3rd-iter.csv").T)[0]

In [32]:
chosen_ws = glob.glob("best-*.csv")
for ws_set_file in chosen_ws:
    ws_set = np.array(pd.read_csv(ws_set_file).T)[0]
#     print ws_set_file
    LAG = int(ws_set_file[9:11])
    to_validate = prediction.get_predictions(LAG, 9, 10, real, ws_set)
    print "LAG %d"%LAG,validate.get_validations(LAG, to_validate, real)

LAG 23 [205.82624824351581, 374.86547851071168, 726.02637891644849, 1284.8735497820055, 1627.4597425170004]
LAG 21 [166.85156662968512, 292.0898130079363, 564.366573591867, 1035.0747400189584, 1527.0827702830245]
LAG 22 [168.91992707974032, 294.43310789395724, 581.26512237641293, 1094.9170235354613, 1583.9765963384611]
LAG 19 [166.58144315243547, 288.90438923758734, 543.77910579020795, 947.7903870066549, 1391.3705312824545]


In [147]:
# # pd.DataFrame(best_param[2]).to_csv("best-for-18-lags-ws.csv",index=False)

# lg = [16,17,18,20,22]
# a = np.array(pd.read_csv("best-for-18-lags-ws.csv").T)[0]

# # for i in lg:
# #     if i == len(a):
# #         arr = a
# #         print len(arr)
# #     elif i < len(a):
# #         arr = a[:-(18-i)]
# #         print len(arr)
# #     else:
# #         mean_val = 1/float(i)
# #         arr = np.append(a,[mean_val]*(i-len(a)))
# #         print arr
        
# def ws_helper(LAG, ws, 18):
#     n_ws = len(ws)
#     if LAG == n_ws:
#         print "eq"
#         arr = ws[:LAG+1]
#     elif LAG < n_ws:
#         print "les"
#         arr = ws[:-(n_ws - LAG + 1)]
#     else:
#         print "else"
#         tail = (LAG - n_ws) - 1
#         mean_val = 1/(float(LAG)*tail)
#         arr = np.append(ws,[mean_val]*tail)
#     return arr

# len(ws_helper(20,a))

les


19

In [69]:
%%time
[compute_ret(temp.get_alphas(i,9,10)) for i in range(16,26)]

NameError: name 'compute_ret' is not defined

In [10]:
print "im done"

im done
